In [1]:
from gensim import corpora, models, similarities
import jieba
import pickle
import pandas as pd

In [2]:
path_df = "/content/rights-dataframe.pickle"
with open(path_df, 'rb') as data:
    category_df = pickle.load(data)

In [3]:
# pieceName has .txt at the end, code to remove it  
for index, row in category_df.iterrows():
  row['pieceName']=row['pieceName'][:-4]

In [4]:
# Common not meaningful names are removed
for index, row in category_df.iterrows():
  if row['pieceName']=="1-Short title" or row['pieceName']=="0-Intro":
    row['pieceName']=''

In [5]:
# next line charachter is removed 
category_df['content'] = category_df['content'].str.replace("\n", " ")

In [6]:
category_df

,category,legName,pieceName,content
0,rights,Matrimonial Rights and Inheritance Ordinance,,This Ordinance may be cited as the Matrimonial...
1,rights,Matrimonial Rights and Inheritance Ordinance,,This Ordinance may be cited as the Matrimonial...
2,rights,Matrimonial Rights and Inheritance Ordinance,vol,"Chapter 57, Volume No.3 Page No.17"
3,rights,Matrimonial Rights and Inheritance Ordinance,36-The rules of the Roman-Dutch law as it prev...,In all questions relating to the distribution ...
4,rights,Matrimonial Rights and Inheritance Ordinance,35-Collation by children or grand -children,Children or grandchildren by representation be...
...,...,...,...,...
157,rights,Fundamental Rights in The Constitution of the ...,"14-Freedom of speech, assembly, association, o...",(1) Every citizen is entitled to – (a) the fre...
158,rights,Fundamental Rights in The Constitution of the ...,14A-Right of access to information,(1) Every citizen shall have the right of acce...
159,rights,Fundamental Rights in The Constitution of the ...,15-Restrictions on fundamental rights,(1) The exercise and operation of the fundamen...
160,rights,Fundamental Rights in The Constitution of the ...,16-Existing written law and unwritten law to c...,(1) All existing written law and unwritten law...


In [7]:
#doucmnets should be ranked according legName, pieceName,content  
data=[]
for index, row in category_df.iterrows():
  full_content ='' 
  full_content= row['legName']+" "+row['pieceName']+" "+row['content']
  data.append(full_content)

In [8]:
# dictionary - convert search word to vector 
# jieba - cuts the kewords into segments 
data = [jieba.lcut(text) for text in data]
dictionary = corpora.Dictionary(data)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.817 seconds.
Prefix dict has been built successfully.


In [9]:
# counts the word features of the dictionary of texts
feature_cnt = len(dictionary.token2id)
# counts the number of occurrences of a word within the corpus
corpus = [dictionary.doc2bow(text) for text in data]  
# Creates a tf-idf model
tfidf = models.TfidfModel(corpus)   
# Convert and index the corpus to check for similarity 
matrix = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features = feature_cnt)  

In [10]:
with open('/content/rights_matrix.pickle', 'wb') as output:
    pickle.dump(matrix, output)

In [11]:
with open('/content/rights_dic.pickle', 'wb') as output:
    pickle.dump(dictionary, output)

In [12]:
with open('/content/rights_tfdif.pickle', 'wb') as output:
    pickle.dump(tfidf, output)